In [1]:
import datetime as dt
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/step3_features_summary.csv', parse_dates=['Time'], index_col='Time')
df.head()

,GDP_growth,Spending_to_GDP_diff,Unemployment_diff,Rent_income_diff,Inflation_diff,PPI_diff_2
Time,,,,,,
1997-12-31,4.487859,-1.155073,-0.7,-0.016114,1.25167,-1.94017
1998-03-31,4.855286,-1.078558,-0.5,-0.024101,0.98445,-2.10186
1998-06-30,4.095967,-1.002043,-0.5,-0.027875,1.08290,-0.88027
1998-09-30,4.097735,-0.925527,-0.3,-0.025923,1.08290,-0.55690
1998-12-31,4.879111,-0.849012,-0.3,-0.016616,1.05477,-0.26948


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94 entries, 1997-12-31 to 2021-03-31
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   GDP_growth            94 non-null     float64
 1   Spending_to_GDP_diff  94 non-null     float64
 2   Unemployment_diff     94 non-null     float64
 3   Rent_income_diff      94 non-null     float64
 4   Inflation_diff        94 non-null     float64
 5   PPI_diff_2            94 non-null     float64
dtypes: float64(6)
memory usage: 5.1 KB


In [4]:
df.describe()

,GDP_growth,Spending_to_GDP_diff,Unemployment_diff,Rent_income_diff,Inflation_diff,PPI_diff_2
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,2.091029,0.061808,0.118085,-0.010401,1.800067,-0.068036
std,2.163947,1.066278,1.392064,0.024057,0.962070,4.748891
min,-9.032775,-1.819629,-1.400000,-0.136863,-1.501860,-18.952680
25%,1.504706,-0.559768,-0.600000,-0.022274,1.263270,-2.061437
50%,2.324297,-0.091719,-0.300000,-0.010085,1.802350,0.035915
75%,3.296804,0.438832,0.300000,0.001084,2.324028,2.506140
max,5.297738,3.439162,7.500000,0.046785,4.658890,11.928510


### Test for stationarity
Check stationarity of multivariate time series data with Johansen's test.

In [5]:
coint_johansen(df, -1, 1).eig

array([0.40882277, 0.32857204, 0.1302609 , 0.11621769, 0.09402035,
       0.02159378])

### Train/test split

In [6]:
train, test = df[:int(0.8*len(df['GDP_growth']))], df[int(0.8*len(df['GDP_growth'])):]

In [7]:
model = VAR(df)
results = model.fit(maxlags=12, ic='aic')
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 05, Jul, 2021
Time:                     15:16:05
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                   -19.7675
Nobs:                     82.0000    HQIC:                  -27.4617
Log likelihood:           1077.42    FPE:                4.06481e-12
AIC:                     -32.6229    Det(Omega_mle):     8.91113e-14
--------------------------------------------------------------------
Results for equation GDP_growth
                              coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------------
const                            0.677375         4.354539            0.156           0.876
L1.GDP_growth                   -2.014375         1.528005           -1.318           0.187
L1.Spending

In [8]:
import joblib
filepath = 'models/var_model.pkl'
joblib.dump(results, filepath)

['models/var_model.pkl']

In [9]:
model = VAR(endog=train)
model = model.fit()
prediction = model.forecast(model.y, steps=len(test))

In [10]:
df_pred = pd.DataFrame(df, columns=df.columns)
df_pred.head()

,GDP_growth,Spending_to_GDP_diff,Unemployment_diff,Rent_income_diff,Inflation_diff,PPI_diff_2
Time,,,,,,
1997-12-31,4.487859,-1.155073,-0.7,-0.016114,1.25167,-1.94017
1998-03-31,4.855286,-1.078558,-0.5,-0.024101,0.98445,-2.10186
1998-06-30,4.095967,-1.002043,-0.5,-0.027875,1.08290,-0.88027
1998-09-30,4.097735,-0.925527,-0.3,-0.025923,1.08290,-0.55690
1998-12-31,4.879111,-0.849012,-0.3,-0.016616,1.05477,-0.26948


In [12]:
for i in df_pred.columns:
    print('RMSE for', i, 'is:', sqrt(mean_squared_error(prediction[i], test[i])))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices